In [1]:
# 1. Instala dependencias
!pip install torch torchvision pillow

In [2]:
# 2. Descarga el modelo
!mkdir -p ckpts
!wget https://dl.fbaipublicfiles.com/videoseal/y_256b_img.jit -P ckpts/

--2025-10-28 15:00:02--  https://dl.fbaipublicfiles.com/videoseal/y_256b_img.jit
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.37.123, 13.35.37.90, 13.35.37.84, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.37.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228524815 (218M) [binary/octet-stream]
Saving to: ‘ckpts/y_256b_img.jit’

y_256b_img.jit      100%[===================>] 217.94M   217MB/s    in 1.0s    

2025-10-28 15:00:03 (217 MB/s) - ‘ckpts/y_256b_img.jit’ saved [228524815/228524815]



In [3]:
# 3. Sube la imagen que quieres verificar
from google.colab import files
uploaded = files.upload()  # Selecciona tu imagen, por ejemplo "mi_imagen_watermarked.jpg" o cualquier otra

Saving mi_imagen_watermarked.jpg to mi_imagen_watermarked.jpg


In [5]:
# 4. Verifica si tiene watermark
from PIL import Image
import torch
from torchvision.transforms.functional import to_tensor

def text_to_binary_tensor(text, bits=256):
    # Convierte texto a binario y rellena hasta 'bits' bits
    binary = ''.join(format(ord(c), '08b') for c in text)
    binary = binary.ljust(bits, '0')[:bits]
    arr = [int(b) for b in binary]
    return torch.tensor(arr).float().unsqueeze(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load("ckpts/y_256b_img.jit").to(device).eval()

# Cambia por el nombre del archivo que subiste
nombre_imagen = list(uploaded.keys())[0]
img_check = to_tensor(Image.open(nombre_imagen)).unsqueeze(0).to(device)

# Detecta el watermark
preds = model.detect(img_check)

# Convierte el resultado de la detección y el original a bits
bits_detectados = (preds > 0).int()
msg_original = text_to_binary_tensor("lvrpiz").to(device)
bits_original = msg_original.int()

# Asegúrate de que ambos tengan la misma shape
bits_detectados = bits_detectados[:, :256]  # Solo los primeros 256 bits

coincidencias = (bits_detectados == bits_original).sum().item()
print(f"Bits correctos: {coincidencias}/256")

if coincidencias > 200:
    print("¡La imagen contiene el watermark 'lvrpiz'!")
else:
    print("La imagen NO contiene el watermark 'lvrpiz'.")

Bits correctos: 232/256
¡La imagen contiene el watermark 'lvrpiz'!
